<a href="https://colab.research.google.com/github/kanishka55/hj/blob/main/wordEmbedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [116]:
from google.colab import drive
drive.mount ('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [117]:
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from pandas.io.parsers import read_csv
from nltk.tokenize import word_tokenize
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import NMF
from time import time
from unicodedata import category
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
import pandas as pd
#import pyLDAvis.sklearn 
import re
import matplotlib.pyplot as plt
import numpy as np
import string
from wordcloud import WordCloud
from itertools import combinations
import gensim 
import csv
from sklearn.preprocessing import normalize 
import statistics
import warnings
warnings.filterwarnings('ignore')  # To ignore all warnings that arise here to enhance clarity
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
import gensim
from gensim.models import Word2Vec
from sklearn import cluster
from sklearn import metrics

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [118]:
from nltk.corpus.reader import sentiwordnet
class ContentModeling:
        
    def loadData(self, fields, inputfile):
        
        articles=[]
        actual=[]
           
      
        df=read_csv(inputfile, encoding = "ISO-8859-1").astype(str)
     
        for index, row in df.iterrows():
            if len(row) > 1 :
                actual.append(row[0])
                articles.append(row[1])  
            else:
                articles.append(row[0])
        del df
        
        return articles,actual

    def prePros(self,text):
        """ Tokenize text and lemmatize words removing punctuation """
        re_url = re.compile(r'(?:http|ftp|https)://(?:[\\w_-]+(?:(?:\\.[\\w_-]+)+))(?:[\\w.,@?^=%&:/~+#-]*[\\w@?^=%&/~+#-])?')
        re_email = re.compile(r'[a-zA-Z][\w\.-]*[a-zA-Z0-9]@([a-zA-Z0-9][\w\.-]*[a-zA-Z0-9]\.[a-zA-Z][a-zA-Z\.]*[a-zA-Z])')
        re2_email = re.compile(r'(?:[a-z0-9!#$%&*+/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&*+/=?^_`{|}~-]+)*|"(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21\x23-\x5b\x5d-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])*")@(?:(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\.)+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?|\[(?:(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9]))\.){3}(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9])|[a-z0-9-]*[a-z0-9]:(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21-\x5a\x53-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])+)\])')

        # remove links
        text = re.sub(r"http\S+", "", str(text))
        # remove special chars and numbers
        #text = re.sub("[^A-Za-z]+", " ", text)
        text = re.sub(r'(RT[^\n]+\n)', '', text)
        text = re.sub(re_url, '', text)
        text = re.sub(re_email, '', text)
        text = re.sub(re2_email, '', text)
        text = re.sub(r'\d+', '', text)
        exclude = list(set(string.punctuation)) 
        text = ''.join(ch for ch in text if ch not in exclude)#delete punctuation
        text = ''.join(ch for ch in text if category(ch)[0] != 'P') #delete punctuation ASCII as well as other
        text = text.lower()

        stop_words = stopwords.words("english")
        stop_words.extend(["rt","shouldnt","ya","yous","youve","youu","pornandroidiphoneipadsexxxx"]) # additional words to remove
        word_tokens = word_tokenize(text)
        filtered_text = [word for word in word_tokens if word not in stop_words]
        
        return filtered_text


    def articlestoSentence(self,art):
      sentences=[]
      for i in art:
        sentences.append(d.prePros(i))
      return sentences

    def word2VecModel(self,sent):
      model = Word2Vec(sent, min_count=1)
      return model


    def sent_vectorizer(self,sent,model):
      sent_vec =[]
      numw = 0
      for w in sent:
        try:
          if numw == 0:
            sent_vec = model[w]
          else:
            sent_vec = np.add(sent_vec, model[w])
            numw+=1
        except:
            pass
     
      return np.asarray(sent_vec) / numw

    def word2VecVector(self,sent,model):
      model=model
      X=[]
      for sentence in sent:
        X.append(d.sent_vectorizer(sentence,model))

      return X


In [119]:
d=ContentModeling()

In [120]:
no_topics = 3 # number of topics you want to identify
no_top_words = 15  # number of words to represent - making this too large will add low pobable words
inputfile = '/content/drive/MyDrive/Colab Notebooks/labeled_data2.csv' #csv file containing short text : youtube.csv, twitter.csv or timesplitted datafiles
fields=['class','tweet'] # column names of the csv file where text resides : 'text', 'description', 'title', 'earliest_block_message','claimant', 'earliest_block_long_message','earliest_block_short_message'


In [121]:
articles,actual=d.loadData(fields,inputfile)
sentences = d.articlestoSentence(articles)
model = d.word2VecModel(sentences)
X = d.word2VecVector(sentences,model)

In [122]:
sentences[0:10]

[['mayasolovely',
  'woman',
  'complain',
  'cleaning',
  'house',
  'amp',
  'man',
  'always',
  'take',
  'trash'],
 ['mleew',
  'boy',
  'dats',
  'coldtyga',
  'dwn',
  'bad',
  'cuffin',
  'dat',
  'hoe',
  'st',
  'place'],
 ['urkindofbrand',
  'dawg',
  'sbabylife',
  'ever',
  'fuck',
  'bitch',
  'start',
  'cry',
  'confused',
  'shit'],
 ['cganderson', 'vivabased', 'look', 'like', 'tranny'],
 ['shenikaroberts',
  'shit',
  'hear',
  'might',
  'true',
  'might',
  'faker',
  'bitch',
  'told'],
 ['tmadisonx',
  'shit',
  'blows',
  'meclaim',
  'faithful',
  'somebody',
  'still',
  'fucking',
  'hoes'],
 ['brighterdays',
  'sit',
  'hate',
  'another',
  'bitch',
  'got',
  'much',
  'shit',
  'going'],
 ['selfiequeenbri',
  'cause',
  'im',
  'tired',
  'big',
  'bitches',
  'coming',
  'us',
  'skinny',
  'girls'],
 ['amp', 'might', 'get', 'bitch', 'back', 'amp', 'thats'],
 ['rhythmixx', 'hobbies', 'include', 'fighting', 'mariam', 'bitch']]

In [123]:
X[0:10]

[array([-inf,  inf,  inf,  inf,  inf,  inf,  inf, -inf, -inf, -inf, -inf,
        -inf, -inf, -inf, -inf, -inf, -inf, -inf,  inf, -inf, -inf,  inf,
         inf, -inf,  inf, -inf, -inf,  inf,  inf, -inf,  inf,  inf, -inf,
        -inf,  inf,  inf, -inf,  inf, -inf, -inf, -inf,  inf, -inf, -inf,
         inf,  inf, -inf, -inf, -inf, -inf,  inf,  inf,  inf,  inf,  inf,
         inf,  inf,  inf, -inf, -inf, -inf,  inf, -inf, -inf, -inf, -inf,
        -inf, -inf,  inf,  inf,  inf,  inf,  inf,  inf, -inf, -inf,  inf,
        -inf,  inf,  inf,  inf, -inf,  inf,  inf,  inf,  inf, -inf, -inf,
         inf,  inf, -inf,  inf, -inf,  inf,  inf, -inf, -inf, -inf, -inf,
         inf], dtype=float32),
 array([-inf,  inf,  inf,  inf,  inf,  inf,  inf, -inf, -inf, -inf, -inf,
        -inf, -inf, -inf, -inf, -inf, -inf, -inf,  inf, -inf, -inf,  inf,
         inf, -inf,  inf, -inf, -inf,  inf,  inf, -inf,  inf,  inf, -inf,
        -inf,  inf,  inf, -inf,  inf, -inf, -inf, -inf,  inf, -inf, -inf,
       